# NN 2.0

## Defining data functions

Import needed libraries

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import tensorflow.keras as ks
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

2022-01-14 10:43:51.488007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-14 10:43:51.488033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Get data from file and remove columns with text

In [2]:
def get_swissvotes_data(now:int=651, since:int=299)->pd.DataFrame:
    import re
    
    dataset = pd.read_csv("../data/formatted/swissvotes_edited_nov_2021.csv", sep=';')
    
    regex = re.compile("pdev_.*")
    to_excl = list(filter(regex.match, dataset.columns))
    
    dataset.drop(columns=to_excl, inplace=True)
    dataset.drop(columns=["legisjahr"], inplace=True)
    dataset.drop(columns=["titel_kurz_d", "titel_kurz_f", "titel_off_d", "titel_off_f", "stichwort"], inplace=True)
    dataset.drop(columns=["swissvoteslink", "anzahl", "anneepolitique", "bkchrono_de", "bkchrono_fr"], inplace=True)
    dataset.drop(columns=["curiavista_de", "curiavista_fr", "urheber", "bkresults_de", "bkresults_fr"], inplace=True)
    dataset.drop(columns=["bfsmap_de", "bfsmap_fr", "nach_cockpit_d", "nach_cockpit_f", "nach_cockpit_e"], inplace=True)
    dataset = dataset[dataset["anr"] < now] # we don't care about future votes
    dataset = dataset[dataset["anr"] >= since]
    
    return dataset
print(f"Defined {get_swissvotes_data}")

Defined <function get_swissvotes_data at 0x7f8b0466b670>


In [3]:
def get_left_right_data(now:int=651, since:int=299)->pd.DataFrame:
    col_names = ["anr", "datum", "titel_kurz_d", "ja_proz", "links_rechts", "kons_prog"]
    dataset = pd.read_csv("../data/formatted/brj_lire_konslib_edited.csv", sep=',')
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) < now]
    dataset = dataset.replace("#ZAHL!", np.nan).replace("Nan", np.nan)
    dataset = dataset[dataset["anr"].map(lambda x: x.replace(',', '.')).astype(float) >= since]
    
    return pd.DataFrame(dataset.drop(columns=["titel_kurz"]), index=get_swissvotes_data(now=now, since=since).index)

print(f"Defined {get_left_right_data}")

Defined <function get_left_right_data at 0x7f8b0466b3a0>


In [4]:
def get_l_r_onehot(data:pd.DataFrame=get_left_right_data()["li_re"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["l3", "l2", "l1", "nlr", "r1", "r2", "r3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_l_r_onehot}")

Defined <function get_l_r_onehot at 0x7f8b0466bd30>


In [5]:
def get_kons_prog_onehot(data:pd.DataFrame=get_left_right_data()["kons_prog"])->pd.DataFrame:
    temp = data.replace(np.nan, 4).astype(int)
    temp += 3
    tensor = tf.one_hot(temp, 8).numpy()
    
    temp = pd.DataFrame(tensor, columns=["k3", "k2", "k1", "nkp", "p1", "p2", "p3", "del"], index=data.index)
    return temp.drop(columns=["del"])

print(f"Defined {get_kons_prog_onehot}")

Defined <function get_kons_prog_onehot at 0x7f8b02f180d0>


In [6]:
def get_rechtsform_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["rechtsform"], 5).numpy();
    result = pd.DataFrame(tensor, columns=["ref_obl", "ref_fak", "initiative", "gegen_entw", "stichfr"], index=data.index)
    
    return result.astype(int)
print(f"Defined {get_rechtsform_onehot}")

Defined <function get_rechtsform_onehot at 0x7f8b04631790>


In [7]:
def get_politikbereich_multihot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    polber = data[["d1e1", "d2e1", "d3e1"]]
    polber = polber.replace('.', 0)
    polber = polber.astype(int)
    
    # the names of the columns (they're a bit long)
    cols = ["Staatsordnung", "Aussenpolitik", "Sicherheitspolitik", "Wirtschaft"]
    cols += ["Landwirtschaft", "Öffentliche Finanzen", "Energie", "Verkehr und Infrastruktur"]
    cols += ["Umwelt und Lebensraum", "Sozialpolitik", "Bildung und Forschung", "Kultur, Religion, Medien"]
    
    result = pd.DataFrame(columns=cols, index = data.index)
    for i in range(len(result)):
        row = np.zeros(12)
        for p in polber.iloc[i]:
            if p != 0:
                row[p-1] = 1
        result.iloc[i] = row
    return result.astype(int)

print(f"Defined {get_politikbereich_multihot}")

Defined <function get_politikbereich_multihot at 0x7f8b028b7ca0>


In [8]:
def get_department_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    dep_single = data["dep"].replace('.', 2) # voting at age 18 is the only vote with a '.' and it's dep of inner
    dep_single = dep_single.astype(int)
    tensor = tf.one_hot(dep_single, 8).numpy()
    result = pd.DataFrame(tensor, columns=["EDA", "EDI", "EJPD", "VBS", "EFD", "WBF", "UVEK", "BK"], index=data.index)
    
    return result.astype(int)

print(f"Defined {get_department_onehot}")

Defined <function get_department_onehot at 0x7f8b045f8820>


In [9]:
def get_bundesrat_onehot(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    tensor = tf.one_hot(data["br_pos"].replace('.', 3).astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_bund", "Dagegen_bund", "Keine_bund"], index=data.index)
    return result.astype(int)

print(f"Defined {get_bundesrat_onehot}")

Defined <function get_bundesrat_onehot at 0x7f8b028b7670>


In [10]:
def get_legislatur(low:int=1, high:int=10, data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    leg = data["legislatur"]
    def my_map(x:int, x_min:int=leg.min(0), x_max:int=leg.max(0), y_min:int=low, y_max:int=high)->float:
        return (x-x_min)/(x_max-x_min)*(y_max-y_min)+y_min
    
    normalized = data[["legislatur"]].applymap(my_map)
    return normalized

print(f"Defined {get_legislatur}")

Defined <function get_legislatur at 0x7f8b02e8fee0>


In [11]:
def get_parlament_onehot(data:pd.DataFrame = get_swissvotes_data()["nr_pos"])->pd.DataFrame:
    tensor = tf.one_hot(data.astype(int), 3).numpy()
    result = pd.DataFrame(tensor, columns=["Für_"+data.name, "Dagegen_"+data.name, "Keine_"+data.name], index=data.index)
    return result.astype(int)

print(f"Defined {get_parlament_onehot}")

Defined <function get_parlament_onehot at 0x7f8b0435ef70>


In [12]:
def get_parties(data:pd.DataFrame = get_swissvotes_data())->list:
    import re
    
    regex_incl = re.compile("p_.*")
    regex_excl = re.compile("p_others_.*")
    
    parties_pre = list(filter(regex_incl.match, data.columns))
    parties = [p for p in parties_pre if not regex_excl.match(p)]
    return parties

print(f"Defined {get_parties}")

Defined <function get_parties at 0x7f8b02e8f3a0>


In [13]:
def normalize_party_reco(data:pd.DataFrame = get_swissvotes_data(), names:list = get_parties())->pd.DataFrame:
    # deal with unwanted values first
    normalized = data[names].replace(".", 0)
    normalized.replace(np.nan, 0, inplace=True)
    normalized = normalized.astype(int)
    normalized.replace([3,4,5,66,9999], 0, inplace=True)
    
    result = pd.DataFrame(index=normalized.index)
    
    for p in names: # go through parties and create one hot encoding
        tensor = tf.one_hot(normalized[p], 3).numpy()
        temp = pd.DataFrame(tensor, columns=[p+"_neutral", p+"_ja", p+"_nein"], index=result.index)
        result = result.join(temp)

    return result.astype(int)
print(f"Defined {normalize_party_reco}")

Defined <function normalize_party_reco at 0x7f8b028a9d30>


In [14]:
def get_vote_result(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    result = data["annahme"].replace('.', 0)
    return result.astype(int)

print(f"Defined {get_vote_result}")

Defined <function get_vote_result at 0x7f8b04106820>


In [15]:
# creates a dataframe 
def get_canton_results(data:pd.DataFrame = get_swissvotes_data())->pd.DataFrame:
    import re
    regex = re.compile(".*_annahme")
    canton_names = list(filter(regex.match, data.columns))
    return data[canton_names].replace('.', 0).astype(int)

print(f"Defined {get_canton_results}")

Defined <function get_canton_results at 0x7f8b028a9700>


In [16]:
def get_volk_proz(data:pd.DataFrame=get_swissvotes_data())->pd.DataFrame:
    temp = data["volkja_proz"]
    return temp / 100

print(f"Defined {get_volk_proz}")

Defined <function get_volk_proz at 0x7f8b02fb8e50>


In [17]:
def get_canton_proz(data:pd.DataFrame=get_swissvotes_data(), name:str=None)->pd.DataFrame:
    if name != None:
        return data[name+"_japroz"].replace('.', 50).astype(float) / 100
    
    import re
    regex = re.compile(".._japroz")
    names = list(filter(regex.match, data.columns))
    return data[names].replace('.', 50).astype(float) / 100

print(f"Defined {get_canton_proz}")

Defined <function get_canton_proz at 0x7f8b01d138b0>


In [18]:
def get_canton_names()->list:
    import re
    regex = re.compile(".._japroz")
    return [x[0:2] for x in list(filter(regex.match, get_swissvotes_data().columns))]

print(f"Defined {get_canton_names}")

Defined <function get_canton_names at 0x7f8b01d13160>


## Training the net

In [19]:
# get data    
# the possible inputs for the neural net are:
    # Rechtsform (one hot),
    # Politikbereich (multi hot),
    # Department (one hot),
    # Position of the Bundesrat (one hot),
    # legislatur (normalized from 1-10),
    # Position of Nationalrat (one hot),
    # Position of Ständerat (one hot),
    # Party recommendations (one hot),
    # left, right rating (one hot),
    # conservative, liberal (one hot)
# the possible outputs are:
    # result of the votes (binary),
    # result on a canton level (binary)
    # yes percentage of the people ([0:1]),
    # yes percentage by canton ([0:1])
def get_data(input_func:list=[get_rechtsform_onehot,
                             get_politikbereich_multihot, 
                             get_department_onehot, 
                             get_bundesrat_onehot,
                             get_legislatur,
                             get_parlament_onehot,
                             get_parlament_onehot,
                             normalize_party_reco,
                             get_l_r_onehot,
                             get_kons_prog_onehot], 
            input_args:list=[None, 
                            None,
                            None,
                            None,
                            None,
                            None,
                            get_swissvotes_data()["sr_pos"],
                            None,
                            None,
                            None],
            output_func:list=[get_vote_result,
                             get_canton_results,
                             get_volk_proz, 
                             get_canton_proz],
            output_args:list=[None,
                             None,
                             None,
                             None]):
    input_list = []
    for f, a in zip(input_func, input_args):
        if type(a) == type(None):
            input_list.append(f())
        else:
            input_list.append(f(a))
    
    inputs = pd.concat(input_list, axis=1)
    
    out_list = []
    for f, a in zip(output_func, output_args):
        if type(a) == type(None):
            out_list.append(f())
        else:
            out_list.append(f(a))
    
    outputs = pd.concat(out_list, axis=1)
    
    return get_swissvotes_data(), inputs, outputs

print(f"Defined {get_data}")

Defined <function get_data at 0x7f8b01cf1f70>


In [20]:
def create_model(name:str, input_size:int = len(get_data()[1].columns), hidden:list=[100, 50, 20],
                 output_size:int = len(get_data(output_func=[get_vote_result,get_canton_results], 
                                                output_args=[None,None])[2].columns), 
                 activation:str="relu", activation_output:str="sigmoid", 
                 optimizer=ks.optimizers.SGD(learning_rate=0.1), 
                 loss=ks.losses.BinaryCrossentropy(), 
                 metrics:list=[ks.metrics.BinaryAccuracy(), ks.metrics.FalseNegatives()])->ks.models.Sequential:
    model = ks.models.Sequential(name=name)
    
    model.add(ks.layers.Dense(units=input_size, activation=activation, name="Input"))
    
    for i in range(len(hidden)):
        model.add(ks.layers.Dense(units=hidden[i], activation=activation, name="Hidden_"+str(i)))
        model.add(ks.layers.Dropout(rate=.1, name="Dropout_"+str(i)))
        
    model.add(ks.layers.Dense(units=output_size, activation=activation_output, name="Output"))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

print(f"Defined {create_model}")

2022-01-14 10:43:55.596418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-14 10:43:55.596442: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-14 10:43:55.596459: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fancy-mint): /proc/driver/nvidia/version does not exist
2022-01-14 10:43:55.596690: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Defined <function create_model at 0x7f8b04631ca0>


In [21]:
def train_model(model:ks.models.Sequential, 
                inputs:pd.DataFrame=get_data()[1], 
                outputs:pd.DataFrame=get_data(output_func=[get_vote_result,get_canton_results], 
                                              output_args=[None,None])[2], 
                test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True)->tuple:
    
    from sklearn.model_selection import train_test_split as tss
    in_train, in_test, out_train, out_test = tss(inputs, outputs, test_size=test_size)
    
    history = model.fit(x=in_train, y=out_train, batch_size=batch_size, epochs=epochs, shuffle=shuffle)
    
    return history, in_test, out_test

print(f"Defined {train_model}")

Defined <function train_model at 0x7f8b001a8f70>


### Create and Train models

There are two national models. One for binary results and one for exact percentage of yes votes.

In [22]:
optimizer = ks.optimizers.RMSprop(learning_rate=0.01)
epochs = 200

nat_bin = create_model("BinaryModel", optimizer=optimizer, output_size=1)
nat_ext = create_model("ExactModel", optimizer=optimizer, output_size=1, loss=ks.losses.MeanAbsoluteError(), 
                           metrics=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])

history_bin, in_test_bin, out_test_bin = train_model(nat_bin, epochs=epochs,
                                                     outputs=get_data(output_func=[get_vote_result], 
                                                                      output_args=[None])[2])
history_per, in_test_per, out_test_per = train_model(nat_ext, epochs=epochs,
                                                     outputs=get_data(output_func=[get_volk_proz], 
                                                                      output_args=[None])[2])

Epoch 1/200
3/3 [==============================] - 1s 3ms/step - loss: 1.1785 - binary_accuracy: 0.5486 - false_negatives: 47.0000
Epoch 2/200
3/3 [==============================] - 0s 3ms/step - loss: 2.7398 - binary_accuracy: 0.5000 - false_negatives: 23.0000
Epoch 3/200
3/3 [==============================] - 0s 3ms/step - loss: 0.5751 - binary_accuracy: 0.6875 - false_negatives: 32.0000
Epoch 4/200
3/3 [==============================] - 0s 4ms/step - loss: 0.4608 - binary_accuracy: 0.8125 - false_negatives: 10.0000
Epoch 5/200
3/3 [==============================] - 0s 4ms/step - loss: 0.3980 - binary_accuracy: 0.8472 - false_negatives: 11.0000
Epoch 6/200
3/3 [==============================] - 0s 4ms/step - loss: 0.4562 - binary_accuracy: 0.7639 - false_negatives: 21.0000
Epoch 7/200
3/3 [==============================] - 0s 3ms/step - loss: 0.4456 - binary_accuracy: 0.8681 - false_negatives: 5.0000
Epoch 8/200
3/3 [==============================] - 0s 3ms/step - loss: 0.3134 - bina

3/3 [==============================] - 0s 3ms/step - loss: 4.3476e-04 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 63/200
3/3 [==============================] - 0s 4ms/step - loss: 6.7680e-05 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 64/200
3/3 [==============================] - 0s 3ms/step - loss: 2.2258e-05 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 65/200
3/3 [==============================] - 0s 4ms/step - loss: 8.2682e-05 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 66/200
3/3 [==============================] - 0s 3ms/step - loss: 7.6253e-06 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 67/200
3/3 [==============================] - 0s 3ms/step - loss: 3.0961e-05 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 68/200
3/3 [==============================] - 0s 4ms/step - loss: 8.7964e-05 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 69/200
3/3 [======================

3/3 [==============================] - 0s 3ms/step - loss: 0.1086 - binary_accuracy: 0.9444 - false_negatives: 7.0000
Epoch 122/200
3/3 [==============================] - 0s 4ms/step - loss: 0.0532 - binary_accuracy: 0.9653 - false_negatives: 5.0000
Epoch 123/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0427 - binary_accuracy: 0.9792 - false_negatives: 2.0000
Epoch 124/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0279 - binary_accuracy: 0.9861 - false_negatives: 2.0000
Epoch 125/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0066 - binary_accuracy: 0.9931 - false_negatives: 1.0000
Epoch 126/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0056 - binary_accuracy: 0.9931 - false_negatives: 1.0000
Epoch 127/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0055 - binary_accuracy: 0.9931 - false_negatives: 1.0000    
Epoch 128/200
3/3 [==============================] - 0s 4ms/step - loss: 0.0054 - bina

3/3 [==============================] - 0s 3ms/step - loss: 6.4735e-12 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 180/200
3/3 [==============================] - 0s 3ms/step - loss: 8.2362e-15 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 181/200
3/3 [==============================] - 0s 3ms/step - loss: 8.4325e-10 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 182/200
3/3 [==============================] - 0s 3ms/step - loss: 9.7257e-12 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 183/200
3/3 [==============================] - 0s 4ms/step - loss: 7.9605e-09 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 184/200
3/3 [==============================] - 0s 3ms/step - loss: 9.9569e-09 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 185/200
3/3 [==============================] - 0s 3ms/step - loss: 1.3217e-10 - binary_accuracy: 1.0000 - false_negatives: 0.0000e+00
Epoch 186/200
3/3 [===============

3/3 [==============================] - 0s 3ms/step - loss: 0.0539 - mean_squared_error: 0.0051 - mean_absolute_error: 0.0539
Epoch 39/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0906 - mean_squared_error: 0.0126 - mean_absolute_error: 0.0906
Epoch 40/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0497 - mean_squared_error: 0.0040 - mean_absolute_error: 0.0497
Epoch 41/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0620 - mean_squared_error: 0.0063 - mean_absolute_error: 0.0620
Epoch 42/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0526 - mean_squared_error: 0.0050 - mean_absolute_error: 0.0526
Epoch 43/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0546 - mean_squared_error: 0.0050 - mean_absolute_error: 0.0546
Epoch 44/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0447 - mean_squared_error: 0.0036 - mean_absolute_error: 0.0447
Epoch 45/200
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.0322 - mean_squared_error: 0.0018 - mean_absolute_error: 0.0322
Epoch 98/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0401 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0401
Epoch 99/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0374 - mean_squared_error: 0.0025 - mean_absolute_error: 0.0374
Epoch 100/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0372 - mean_squared_error: 0.0023 - mean_absolute_error: 0.0372
Epoch 101/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0281 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0281
Epoch 102/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0299 - mean_squared_error: 0.0020 - mean_absolute_error: 0.0299
Epoch 103/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0351 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0351
Epoch 104/200
3/3 [========================

3/3 [==============================] - 0s 3ms/step - loss: 0.0296 - mean_squared_error: 0.0017 - mean_absolute_error: 0.0296
Epoch 157/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0258 - mean_squared_error: 0.0012 - mean_absolute_error: 0.0258
Epoch 158/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0264 - mean_squared_error: 0.0012 - mean_absolute_error: 0.0264
Epoch 159/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0293 - mean_squared_error: 0.0015 - mean_absolute_error: 0.0293
Epoch 160/200
3/3 [==============================] - 0s 4ms/step - loss: 0.0276 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0276
Epoch 161/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0263 - mean_squared_error: 0.0013 - mean_absolute_error: 0.0263
Epoch 162/200
3/3 [==============================] - 0s 3ms/step - loss: 0.0345 - mean_squared_error: 0.0021 - mean_absolute_error: 0.0345
Epoch 163/200
3/3 [======================

In [23]:
print(nat_bin.summary(), nat_ext.summary())

Model: "BinaryModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (Dense)               (None, 187)               35156     
                                                                 
 Hidden_0 (Dense)            (None, 100)               18800     
                                                                 
 Dropout_0 (Dropout)         (None, 100)               0         
                                                                 
 Hidden_1 (Dense)            (None, 50)                5050      
                                                                 
 Dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 Hidden_2 (Dense)            (None, 20)                1020      
                                                                 
 Dropout_2 (Dropout)         (None, 20)                

In [24]:
res_bin = nat_bin.evaluate(x=in_test_bin, y=out_test_bin)
res_per = nat_ext.evaluate(x=in_test_per, y=out_test_per)

print(f"Binary model results:\t{res_bin}")
print(f"Exact model results:\t{res_per}")

from sklearn.metrics import r2_score
print(f"R2 metric exact:\t{r2_score(nat_ext.predict(in_test_per), out_test_per)}")
print(f"R2 metric binary:\t{r2_score(nat_bin.predict(in_test_bin), out_test_bin)}")

7/7 [==============================] - 0s 1ms/step - loss: 0.0969 - mean_squared_error: 0.0150 - mean_absolute_error: 0.0969
Binary model results:	[2.938145160675049, 0.7962962985038757, 26.0]
Exact model results:	[0.09688123315572739, 0.014977806247770786, 0.09688123315572739]
R2 metric exact:	0.2195199362479996
R2 metric binary:	0.16455420556440647


**Let's try to create a model for each canton.**

In [25]:
def get_per_canton_models(names:list=get_canton_names(), optimizer=ks.optimizers.SGD(learning_rate=0.01), 
                          loss=ks.losses.MeanAbsoluteError(), 
                          metrics:list=[ks.metrics.MeanSquaredError(), ks.metrics.MeanAbsoluteError()])->list:
    result = []
    for c in names:
        result.append((create_model("Canton_"+c, output_size=1, 
                                    optimizer=optimizer, loss=loss, 
                                    metrics=metrics), c))
    
    return result
print(f"Defined {get_per_canton_models}")

Defined <function get_per_canton_models at 0x7f8b01297790>


In [26]:
def train_per_canton_models(models:list=get_per_canton_models(), 
                            inputs:pd.DataFrame=get_data()[1], 
                            test_size:float=0.6, batch_size:int=50, epochs:int=125, shuffle:bool=True)->list:
    result = []
    
    from sklearn.model_selection import train_test_split as tss
    for t in models:
        itr, ite, otr, ote = tss(inputs, get_canton_proz(name=t[1]), test_size=test_size)
        result.append((t[0].fit(x=itr, y=otr, batch_size=batch_size, epochs=epochs, shuffle=shuffle), ite, ote))
        
    return result
        
print(f"Defined {train_per_canton_models}")

Defined <function train_per_canton_models at 0x7f8b012970d0>


In [27]:
def execute_models():
    models = get_per_canton_models()
    hist = train_per_canton_models(models, epochs=150)
    
    evals = []
    for m, h in zip(models, hist):
        evals.append(m[0].evaluate(x=h[1], y=h[2]))
    
    for m, e in zip(models, evals):
        print(f"{m[0].name} has these results:\t{e}")
        
    return models, evals

In [29]:
def print_extreme_cases():
    models, evals = execute_models()
    
    min_abs = ("", 1)
    max_abs = ("", 0)
    min_sqr = ("", 1)
    max_sqr = ("", 0)
    
    for m, e in zip(models, evals):
        if(e[2]<min_abs[1]):
            min_abs = (m[0].name, e[2])
        elif(e[2]>max_abs[1]):
            max_abs = (m[0].name, e[2])
    
        if(e[1]<min_sqr[1]):
            min_sqr = (m[0].name, e[1])
        elif(e[1]>max_sqr[1]):
            max_sqr = (m[0].name, e[1])
        
    print(f"{max_sqr[0]} has the largest mse of\t{max_sqr[1]}")
    print(f"{min_sqr[0]} has the smallest mse of\t{min_sqr[1]}")
    print(f"{max_abs[0]} has the largest mae of\t{max_abs[1]}")
    print(f"{min_abs[0]} has the smallest mae of\t{min_abs[1]}")

In [31]:
print_extreme_cases()

Epoch 1/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1795 - mean_squared_error: 0.0483 - mean_absolute_error: 0.1795
Epoch 2/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1651 - mean_squared_error: 0.0411 - mean_absolute_error: 0.1651
Epoch 3/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1609 - mean_squared_error: 0.0383 - mean_absolute_error: 0.1609
Epoch 4/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1560 - mean_squared_error: 0.0368 - mean_absolute_error: 0.1560
Epoch 5/150
3/3 [==============================] - 0s 4ms/step - loss: 0.1549 - mean_squared_error: 0.0344 - mean_absolute_error: 0.1549
Epoch 6/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1543 - mean_squared_error: 0.0350 - mean_absolute_error: 0.1543
Epoch 7/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1447 - mean_squared_error: 0.0321 - mean_absolute_error: 0.1447
Epoch 8/150
3/3 [========================

3/3 [==============================] - 0s 3ms/step - loss: 0.1066 - mean_squared_error: 0.0179 - mean_absolute_error: 0.1066
Epoch 61/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1175 - mean_squared_error: 0.0206 - mean_absolute_error: 0.1175
Epoch 62/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1151 - mean_squared_error: 0.0213 - mean_absolute_error: 0.1151
Epoch 63/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1172 - mean_squared_error: 0.0207 - mean_absolute_error: 0.1172
Epoch 64/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1189 - mean_squared_error: 0.0222 - mean_absolute_error: 0.1189
Epoch 65/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1128 - mean_squared_error: 0.0203 - mean_absolute_error: 0.1128
Epoch 66/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1177 - mean_squared_error: 0.0216 - mean_absolute_error: 0.1177
Epoch 67/150
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.1021 - mean_squared_error: 0.0164 - mean_absolute_error: 0.1021
Epoch 120/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1059 - mean_squared_error: 0.0173 - mean_absolute_error: 0.1059
Epoch 121/150
3/3 [==============================] - 0s 3ms/step - loss: 0.0938 - mean_squared_error: 0.0150 - mean_absolute_error: 0.0938
Epoch 122/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1000 - mean_squared_error: 0.0150 - mean_absolute_error: 0.1000
Epoch 123/150
3/3 [==============================] - 0s 3ms/step - loss: 0.0982 - mean_squared_error: 0.0149 - mean_absolute_error: 0.0982
Epoch 124/150
3/3 [==============================] - 0s 3ms/step - loss: 0.0936 - mean_squared_error: 0.0130 - mean_absolute_error: 0.0936
Epoch 125/150
3/3 [==============================] - 0s 4ms/step - loss: 0.0976 - mean_squared_error: 0.0148 - mean_absolute_error: 0.0976
Epoch 126/150
3/3 [======================

3/3 [==============================] - 0s 3ms/step - loss: 0.1399 - mean_squared_error: 0.0296 - mean_absolute_error: 0.1399
Epoch 29/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1320 - mean_squared_error: 0.0266 - mean_absolute_error: 0.1320
Epoch 30/150
3/3 [==============================] - 0s 2ms/step - loss: 0.1353 - mean_squared_error: 0.0279 - mean_absolute_error: 0.1353
Epoch 31/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1304 - mean_squared_error: 0.0268 - mean_absolute_error: 0.1304
Epoch 32/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1344 - mean_squared_error: 0.0283 - mean_absolute_error: 0.1344
Epoch 33/150
3/3 [==============================] - 0s 2ms/step - loss: 0.1354 - mean_squared_error: 0.0275 - mean_absolute_error: 0.1354
Epoch 34/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1303 - mean_squared_error: 0.0261 - mean_absolute_error: 0.1303
Epoch 35/150
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.1075 - mean_squared_error: 0.0190 - mean_absolute_error: 0.1075
Epoch 88/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1050 - mean_squared_error: 0.0181 - mean_absolute_error: 0.1050
Epoch 89/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1096 - mean_squared_error: 0.0192 - mean_absolute_error: 0.1096
Epoch 90/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1050 - mean_squared_error: 0.0177 - mean_absolute_error: 0.1050
Epoch 91/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1129 - mean_squared_error: 0.0201 - mean_absolute_error: 0.1129
Epoch 92/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1126 - mean_squared_error: 0.0197 - mean_absolute_error: 0.1126
Epoch 93/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1097 - mean_squared_error: 0.0190 - mean_absolute_error: 0.1097
Epoch 94/150
3/3 [=============================

...

3/3 [==============================] - 0s 3ms/step - loss: 0.1577 - mean_squared_error: 0.0388 - mean_absolute_error: 0.1577
Epoch 55/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1601 - mean_squared_error: 0.0413 - mean_absolute_error: 0.1601
Epoch 56/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1575 - mean_squared_error: 0.0382 - mean_absolute_error: 0.1575
Epoch 57/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1598 - mean_squared_error: 0.0384 - mean_absolute_error: 0.1598
Epoch 58/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1588 - mean_squared_error: 0.0387 - mean_absolute_error: 0.1588
Epoch 59/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1597 - mean_squared_error: 0.0393 - mean_absolute_error: 0.1597
Epoch 60/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1563 - mean_squared_error: 0.0391 - mean_absolute_error: 0.1563
Epoch 61/150
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.1519 - mean_squared_error: 0.0356 - mean_absolute_error: 0.1519
Epoch 114/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1452 - mean_squared_error: 0.0336 - mean_absolute_error: 0.1452
Epoch 115/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1516 - mean_squared_error: 0.0361 - mean_absolute_error: 0.1516
Epoch 116/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1531 - mean_squared_error: 0.0354 - mean_absolute_error: 0.1531
Epoch 117/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1519 - mean_squared_error: 0.0347 - mean_absolute_error: 0.1519
Epoch 118/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1478 - mean_squared_error: 0.0344 - mean_absolute_error: 0.1478
Epoch 119/150
3/3 [==============================] - 0s 4ms/step - loss: 0.1476 - mean_squared_error: 0.0347 - mean_absolute_error: 0.1476
Epoch 120/150
3/3 [======================

3/3 [==============================] - 0s 3ms/step - loss: 0.1681 - mean_squared_error: 0.0390 - mean_absolute_error: 0.1681
Epoch 23/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1660 - mean_squared_error: 0.0413 - mean_absolute_error: 0.1660
Epoch 24/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1680 - mean_squared_error: 0.0402 - mean_absolute_error: 0.1680
Epoch 25/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1625 - mean_squared_error: 0.0386 - mean_absolute_error: 0.1625
Epoch 26/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1677 - mean_squared_error: 0.0395 - mean_absolute_error: 0.1677
Epoch 27/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1601 - mean_squared_error: 0.0362 - mean_absolute_error: 0.1601
Epoch 28/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1649 - mean_squared_error: 0.0403 - mean_absolute_error: 0.1649
Epoch 29/150
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.1396 - mean_squared_error: 0.0289 - mean_absolute_error: 0.1396
Epoch 82/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1473 - mean_squared_error: 0.0330 - mean_absolute_error: 0.1473
Epoch 83/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1368 - mean_squared_error: 0.0286 - mean_absolute_error: 0.1368
Epoch 84/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1393 - mean_squared_error: 0.0303 - mean_absolute_error: 0.1393
Epoch 85/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1417 - mean_squared_error: 0.0295 - mean_absolute_error: 0.1417
Epoch 86/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1442 - mean_squared_error: 0.0313 - mean_absolute_error: 0.1442
Epoch 87/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1360 - mean_squared_error: 0.0280 - mean_absolute_error: 0.1360
Epoch 88/150
3/3 [=============================

3/3 [==============================] - 0s 3ms/step - loss: 0.1272 - mean_squared_error: 0.0248 - mean_absolute_error: 0.1272
Epoch 140/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1263 - mean_squared_error: 0.0249 - mean_absolute_error: 0.1263
Epoch 141/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1284 - mean_squared_error: 0.0264 - mean_absolute_error: 0.1284
Epoch 142/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1235 - mean_squared_error: 0.0245 - mean_absolute_error: 0.1235
Epoch 143/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1236 - mean_squared_error: 0.0250 - mean_absolute_error: 0.1236
Epoch 144/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1223 - mean_squared_error: 0.0243 - mean_absolute_error: 0.1223
Epoch 145/150
3/3 [==============================] - 0s 3ms/step - loss: 0.1246 - mean_squared_error: 0.0237 - mean_absolute_error: 0.1246
Epoch 146/150
3/3 [======================